# 03 - Ticket Classification System

This notebook implements the ticket classification system using LLaMA.
It loads the model configuration from notebook 02 and applies it to real customer support data.

In [2]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

print("=== Ticket Classification System ===")
print("LLaMA-powered classification for customer support tickets")
print()

=== Ticket Classification System ===
LLaMA-powered classification for customer support tickets



In [3]:
# Load LLaMA model configuration from notebook 02
def load_model_config():
    config_path = Path("../outputs/customer_support_model_config.json")
    
    if config_path.exists():
        with open(config_path, 'r') as f:
            config = json.load(f)
        
        print("✅ Model configuration loaded")
        print(f"Model: {config.get('model_name', 'Unknown')}")
        print(f"Categories: {config.get('categories', [])}")
        print(f"Setup complete: {config.get('setup_complete', False)}")
        
        return config
    else:
        raise FileNotFoundError("Model configuration not found. Please run notebook 02 first.")

model_config = load_model_config()

✅ Model configuration loaded
Model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Categories: ['billing', 'technical', 'general_inquiry', 'account', 'complaint', 'compliment']
Setup complete: True


In [4]:
# Load real customer support data for classification
def load_classification_data():
    """Load real customer support data for classification testing"""
    
    # Try processed data first
    processed_path = Path("../data/processed/test_data.csv")
    if processed_path.exists():
        print("Loading processed test data...")
        df = pd.read_csv(processed_path)
        return df.head(20)  # Use 20 tickets for classification
    
    # Fallback to train data
    train_path = Path("../data/processed/train_data.csv")
    if train_path.exists():
        print("Loading sample from training data...")
        df = pd.read_csv(train_path)
        return df.sample(n=20, random_state=42)
    
    # Last resort: raw data
    raw_path = Path("../data/raw/twcs/twcs.csv")
    if raw_path.exists():
        print("Loading sample from raw Twitter data...")
        df = pd.read_csv(raw_path)
        # Filter for customer queries
        df_filtered = df[df['text'].str.len() > 20]
        return df_filtered.sample(n=20, random_state=42)
    
    raise FileNotFoundError("No customer support data found. Please run notebook 01 first.")

classification_data = load_classification_data()
print(f"Loaded {len(classification_data)} tickets for classification")
print(f"Sample ticket: {classification_data.iloc[0]['text'][:100]}...")

Loading processed test data...
Loaded 20 tickets for classification
Sample ticket: en todos los aos que tengo volando con nunca haba tenido una experiencia como la de ayer 24 vuelo 91...


In [7]:
# Initialize LLaMA-powered classification model
class LLaMATicketClassifier:
    def __init__(self, config):
        self.config = config
        self.model_name = config['model_name']
        self.device = config.get('device', 'cpu')
        self.categories = config['categories']
        self.priority_levels = config['priority_levels']
        self.sentiment_types = config['sentiment_types']

        # Load LLaMA setup config for proper model initialization
        llama_config_path = Path("../outputs/llama_setup_config.json")
        if llama_config_path.exists():
            with open(llama_config_path, 'r') as f:
                self.llama_config = json.load(f)
            self.device = self.llama_config['system_specs']['device']

        self.model = None
        self.tokenizer = None

    def setup_model(self):
        """Setup LLaMA model with 12GB optimizations"""
        print(f"Setting up LLaMA model: {self.model_name}")
        print(f"Device: {self.device} (Intel graphics optimized)")

        # Clean memory
        import gc
        gc.collect()

        # Load tokenizer
        print("Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Load model with memory optimizations
        print("Loading model with 12GB optimizations...")
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            dtype=torch.float32,
            low_cpu_mem_usage=True,
            device_map=None
        )
        self.model = self.model.to(self.device)
        self.model.eval()

        print("✅ LLaMA classification model ready")

    def classify_batch(self, tickets):
        """Classify a batch of tickets using LLaMA"""
        results = []

        for i, ticket in enumerate(tickets, 1):
            print(f"Classifying ticket {i}/{len(tickets)}...")
            classification = self.classify_single_ticket(ticket)
            results.append({
                'ticket_text': ticket,
                'category': classification['category'],
                'priority': classification['priority'],
                'sentiment': classification['sentiment'],
                'estimated_hours': classification['estimated_hours']
            })

            if i % 5 == 0:
                import gc
                gc.collect()

        return pd.DataFrame(results)

    def classify_single_ticket(self, ticket_text):
        """Classify a single ticket using hybrid approach"""
        content_classification = self.content_analysis_fallback(ticket_text)

        # If no model loaded, fallback only
        if self.model is None or self.tokenizer is None:
            return content_classification

        try:
            llama_classification = self.llama_classify(ticket_text)
            final = content_classification.copy()

            if llama_classification['category'] != 'general_inquiry':
                final['category'] = llama_classification['category']

            if llama_classification['priority'] != 'medium':
                final['priority'] = llama_classification['priority']

            if llama_classification['sentiment'] != 'neutral':
                final['sentiment'] = llama_classification['sentiment']

            if llama_classification['estimated_hours'] != 2.0:
                final['estimated_hours'] = llama_classification['estimated_hours']

            return final

        except Exception as e:
            print(f"LLaMA classification failed, fallback used: {e}")
            return content_classification

    def llama_classify(self, ticket_text):
        """Use LLaMA for ticket classification"""
        prompt = f"""<|system|>
You are a customer support classifier. Analyze the ticket and respond with exact format.

<|user|>
Ticket: {ticket_text}

Classify this into:
- Category: billing, technical, general_inquiry, account, complaint, or compliment
- Priority: high, medium, or low  
- Sentiment: positive, negative, or neutral
- Hours: estimated resolution time (0.5 to 8.0)

<|assistant|>
Category: """

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=400, truncation=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                inputs['input_ids'],
                max_new_tokens=50,
                temperature=0.2,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        classification_part = response.split("<|assistant|>")[-1].strip()

        return self.parse_llama_output(classification_part)

    def parse_llama_output(self, output_text):
        """Parse LLaMA classification output"""
        result = {
            'category': 'general_inquiry',
            'priority': 'medium',
            'sentiment': 'neutral',
            'estimated_hours': 2.0
        }

        output_lower = output_text.lower()

        # Extract category
        for cat in self.categories:
            if cat in output_lower:
                result['category'] = cat
                break

        # Extract priority
        for priority in self.priority_levels:
            if priority in output_lower:
                result['priority'] = priority
                break

        # Extract sentiment
        for sentiment in self.sentiment_types:
            if sentiment in output_lower:
                result['sentiment'] = sentiment
                break

        # Extract hours using regex
        import re
        hours_match = re.search(r'(\d+(?:\.\d+)?)\s*hours?', output_lower)
        if hours_match:
            try:
                hours = float(hours_match.group(1))
                if 0.1 <= hours <= 48.0:
                    result['estimated_hours'] = hours
            except:
                pass

        return result

    def content_analysis_fallback(self, ticket_text):
        """Fallback content analysis"""
        text_lower = ticket_text.lower()

        billing_words = ['bill', 'billing', 'charge', 'payment', 'invoice', 'refund']
        technical_words = ['error', 'bug', 'crash', 'technical', 'app', 'website']
        account_words = ['account', 'login', 'password', 'reset', 'username']
        complaint_words = ['terrible', 'awful', 'hate', 'worst', 'frustrated']
        compliment_words = ['love', 'great', 'awesome', 'excellent', 'thank']

        scores = {
            'billing': sum(1 for w in billing_words if w in text_lower),
            'technical': sum(1 for w in technical_words if w in text_lower),
            'account': sum(1 for w in account_words if w in text_lower),
            'complaint': sum(1 for w in complaint_words if w in text_lower),
            'compliment': sum(1 for w in compliment_words if w in text_lower)
        }

        max_score = max(scores.values())
        category = 'general_inquiry'
        if max_score > 0:
            category = max(scores, key=scores.get)

        urgent_words = ['urgent', 'emergency', 'critical', 'immediately']
        low_priority_words = ['question', 'how', 'when', 'what']

        priority = 'medium'
        if any(w in text_lower for w in urgent_words):
            priority = 'high'
        elif any(w in text_lower for w in low_priority_words):
            priority = 'low'

        positive_words = ['love', 'great', 'awesome', 'excellent', 'thank', 'good']
        negative_words = ['hate', 'terrible', 'awful', 'bad', 'frustrated', 'problem']

        positive_count = sum(1 for w in positive_words if w in text_lower)
        negative_count = sum(1 for w in negative_words if w in text_lower)

        sentiment = 'neutral'
        if positive_count > negative_count + 1:
            sentiment = 'positive'
        elif negative_count > positive_count + 1:
            sentiment = 'negative'

        base_hours = {
            'billing': 1.5,
            'technical': 3.0,
            'account': 1.0,
            'complaint': 2.0,
            'compliment': 0.5,
            'general_inquiry': 2.0
        }
        multiplier = {'high': 1.5, 'medium': 1.0, 'low': 0.7}

        estimated_hours = base_hours[category] * multiplier[priority]

        return {
            'category': category,
            'priority': priority,
            'sentiment': sentiment,
            'estimated_hours': round(estimated_hours, 1)
        }


print("Initializing LLaMA-powered ticket classifier...")
classifier = LLaMATicketClassifier(model_config)
classifier.setup_model()
print("✅ LLaMA Classifier ready")


Initializing LLaMA-powered ticket classifier...
Setting up LLaMA model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Device: cpu (Intel graphics optimized)
Loading tokenizer...
Loading model with 12GB optimizations...
✅ LLaMA classification model ready
✅ LLaMA Classifier ready


In [8]:
# Run classification on real data
print("Running classification on real customer support tickets...")

# Get tickets as list
tickets_list = classification_data['text'].tolist()

# Classify all tickets
classification_results = classifier.classify_batch(tickets_list)

print(f"\n✅ Classification complete!")
print(f"Processed {len(classification_results)} tickets")

# Show summary
print("\n📊 Classification Summary:")
print(f"Categories: {classification_results['category'].value_counts().to_dict()}")
print(f"Priorities: {classification_results['priority'].value_counts().to_dict()}")
print(f"Sentiments: {classification_results['sentiment'].value_counts().to_dict()}")
print(f"Avg ETA: {classification_results['estimated_hours'].mean():.1f} hours")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Running classification on real customer support tickets...
Classifying ticket 1/20...
Classifying ticket 2/20...
Classifying ticket 3/20...
Classifying ticket 4/20...
Classifying ticket 5/20...
Classifying ticket 6/20...
Classifying ticket 7/20...
Classifying ticket 8/20...
Classifying ticket 9/20...
Classifying ticket 10/20...
Classifying ticket 11/20...
Classifying ticket 12/20...
Classifying ticket 13/20...
Classifying ticket 14/20...
Classifying ticket 15/20...
Classifying ticket 16/20...
Classifying ticket 17/20...
Classifying ticket 18/20...
Classifying ticket 19/20...
Classifying ticket 20/20...

✅ Classification complete!
Processed 20 tickets

📊 Classification Summary:
Categories: {'billing': 15, 'technical': 5}
Priorities: {'high': 20}
Sentiments: {'positive': 20}
Avg ETA: 1.7 hours


In [9]:
# Save classification results
output_dir = Path("../outputs")
output_dir.mkdir(exist_ok=True)

# Save detailed results
classification_results.to_csv(output_dir / 'ticket_classifications.csv', index=False)

# Save summary statistics
summary_stats = {
    'total_tickets': len(classification_results),
    'category_distribution': classification_results['category'].value_counts().to_dict(),
    'priority_distribution': classification_results['priority'].value_counts().to_dict(),
    'sentiment_distribution': classification_results['sentiment'].value_counts().to_dict(),
    'avg_estimated_hours': float(classification_results['estimated_hours'].mean()),
    'classification_system': 'LLaMA-powered'
}

with open(output_dir / 'classification_summary.json', 'w') as f:
    json.dump(summary_stats, f, indent=2)

print("💾 Results saved:")
print(f"- Classifications: {output_dir}/ticket_classifications.csv")
print(f"- Summary: {output_dir}/classification_summary.json")
print("\n🎉 Ticket Classification System Complete!")
print("Ready to proceed to notebook 04 (ETA Prediction)")

💾 Results saved:
- Classifications: ..\outputs/ticket_classifications.csv
- Summary: ..\outputs/classification_summary.json

🎉 Ticket Classification System Complete!
Ready to proceed to notebook 04 (ETA Prediction)
